# BGP Stream

### Task 1: Analysis of BGP Table Statistics
Use PyBGPStream (project=ris-live, exclude from_time and until_time), to preform real-time analysis of the BGP table statistics for AS131072, focusing on the characteristics of advertisements. Print the metrics evaluating the degree of similarity of the AS paths of these specific advertisements to their encompassing aggregate.

In [1]:
import pybgpstream

# Define the target AS
target_as = 131072

# Create a new PyBGPStream instance
stream = pybgpstream.BGPStream(
    project="ris-live",
    record_type="updates",
    filter="type ribs and path " + str(target_as)
)

# Create a new PyASPathAnalyzer instance
analyzer = pybgpstream.PyASPathAnalyzer()

# Dictionary to store the number of advertisements and the similarity metric
advertisements = {}

# Start the stream
for elem in stream:
    # Get the AS path for the current update
    as_path = elem.fields['as-path'].split()
    as_set = set(as_path)

    # Check if the update is an advertisement for the target AS
    if target_as in as_set:
        # Calculate the similarity metric
        similarity_metric = analyzer.calc_similarity(as_path, as_set)

        # Get the prefix for the current update
        prefix = elem.fields['prefix']

        # Store the similarity metric in the dictionary
        advertisements[prefix] = similarity_metric

# Print the advertisements and their similarity metrics
for prefix, similarity_metric in advertisements.items():
    print(f"Prefix: {prefix}, Similarity Metric: {similarity_metric}")

AttributeError: module 'pybgpstream' has no attribute 'PyASPathAnalyzer'

### Task 2: BGP Path Update Analysis/Report
Use PyBGPStream to collect BGP path updates for AS14061 and summarize: origin paths, next-hop paths, transit paths, first-hop changes, and prepend actions.

In [3]:
import pybgpstream

# Create a BGPStream instance
stream = pybgpstream.BGPStream(
    project="routeviews",
    filter="type ribs and path 14061"
)

# Define dictionaries and counters to store summarized information
origin_paths = {}
next_hop_paths = {}
transit_paths = {}
first_hop_changes = 0
prepend_actions = 0

# Start the stream
for elem in stream:
    # Get the AS path
    as_path = elem.fields['as-path'].split(" ")

    # Check if the prefix originated from AS14061
    if as_path[-1] == '14061':
        origin_paths[elem.fields['prefix']] = as_path

    # Store next-hop paths
    next_hop = as_path[-1]
    if next_hop in next_hop_paths:
        next_hop_paths[next_hop] += 1
    else:
        next_hop_paths[next_hop] = 1

    # Check for transit paths
    if len(as_path) > 1:
        transit_paths[elem.fields['prefix']] = as_path

    # Check for first-hop changes
    if as_path[0] != '14061':
        first_hop_changes += 1

    # Check for prepend actions
    if as_path.count('14061') > 1:
        prepend_actions += 1

# Print summarized information
print("Origin Paths:")
for prefix, path in origin_paths.items():
    print(f"Prefix: {prefix}, AS Path: {' -> '.join(path)}")

print("\nNext-Hop Paths:")
for next_hop, count in next_hop_paths.items():
    print(f"Next Hop: {next_hop}, Number of Prefixes: {count}")

print("\nTransit Paths:")
for prefix, path in transit_paths.items():
    print(f"Prefix: {prefix}, AS Path: {' -> '.join(path)}")

print(f"\nNumber of First-Hop Changes: {first_hop_changes}")
print(f"Number of Prepend Actions: {prepend_actions}")


KeyboardInterrupt: 

### Task 3: BGP Path Changes
Use PyBGPStream to access live data stream (project='ris-live') and review a series of BGP update messages that show alterations in the AS_PATH attribute for key routes managed by AS14061. For each identified AS_PATH alteration, the script should detail the change, including the previous and updated AS_PATH, the affected prefix, and the timestamp of the update. 

In [4]:
import pybgpstream

# Create a BGPStream instance
stream = pybgpstream.BGPStream(
    project="ris-live",
    filter="type updates and path 14061"
)

# Dictionary to store the previous AS_PATH for each prefix
previous_as_paths = {}

# Start the stream
for elem in stream:
    # Get the prefix and AS_PATH
    prefix = elem.fields["prefix"]
    as_path = elem.fields["as-path"]

    # Check if this prefix has been seen before
    if prefix in previous_as_paths:
        previous_as_path = previous_as_paths[prefix]

        # Check if AS_PATH has been altered
        if as_path != previous_as_path:
            # Print details of the alteration
            print(f"Prefix: {prefix}")
            print(f"Previous AS_PATH: {previous_as_path}")
            print(f"Updated AS_PATH: {as_path}")
            print(f"Timestamp: {elem.time}")
            print()

    # Update the previous AS_PATH for this prefix
    previous_as_paths[prefix] = as_path

Prefix: 64.68.236.0/22
Previous AS_PATH: 14061 6453 3356 13904 13904 13904 13904 13904 13904
Updated AS_PATH: 14061 3356 13904 13904 13904 13904 13904 13904
Timestamp: 1707807680.59

Prefix: 45.188.132.0/24
Previous AS_PATH: 14061 1299 2914 52941 269544
Updated AS_PATH: 14061 2914 52941 269544
Timestamp: 1707807680.57

Prefix: 2605:dd40:9005::/48
Previous AS_PATH: 14061 6453 2914 137409 23961 398549
Updated AS_PATH: 14061 1299 2914 2914 2914 2914 137409 23961 398549
Timestamp: 1707807680.58

Prefix: 2607:ff00:b00::/40
Previous AS_PATH: 14061 2914 137409 12189 400672
Updated AS_PATH: 14061 6461 137409 12189 400672
Timestamp: 1707807680.58

Prefix: 176.124.58.0/24
Previous AS_PATH: 14061 2914 3356 9002 12310 44679 209856
Updated AS_PATH: 14061 3356 174 12310 44679 209856
Timestamp: 1707807680.53

Prefix: 2602:fb2a:ff::/48
Previous AS_PATH: 14061 3356 6453 16509 14618 213151
Updated AS_PATH: 14061 2914 174 16509 14618 213151
Timestamp: 1707807680.53

Prefix: 138.121.148.0/22
Previous AS_P

KeyboardInterrupt: 

### Task 4: AS Announcement Trend Reporting

Create a PyBGPStream script that generates reports on the time trend of the total span of address space advertised by AS7296. The script should collect data from BGP announcements from 'rrc00', calculate the span of address space from 02-03-2024 13:00 to 14:59:59, and aggregate this data into a time-series report. Do not add ASN in 'filter' parameter.

In [ ]:
import pybgpstream
from datetime import datetime

# Create a BGPStream instance
stream = pybgpstream.BGPStream(
    project="routeviews",
    filter="type ribs and collector rrc00"
)

# Define start and end times
start_time = datetime.strptime("2024-02-03 13:00:00", "%Y-%m-%d %H:%M:%S")
end_time = datetime.strptime("2024-02-03 14:59:59", "%Y-%m-%d %H:%M:%S")

# Dictionary to store the span of address space at each timestamp
address_space_span = {}

# Start the stream
for elem in stream:
    # Convert timestamp to datetime object
    timestamp = datetime.utcfromtimestamp(elem.time)

    # Check if the timestamp is within the specified range
    if start_time <= timestamp <= end_time:
        # Get the prefix length
        prefix_length = int(elem.fields['prefix'].split("/")[1])

        # Calculate the address space span
        address_space_span[timestamp] = 2**(32 - prefix_length)

# Aggregate data into a time-series report
total_address_space = 0
time_series_report = []
for timestamp in sorted(address_space_span.keys()):
    total_address_space += address_space_span[timestamp]
    time_series_report.append((timestamp, total_address_space))

# Print the time-series report
print("Time\t\t\tTotal Address Space")
for timestamp, total_space in time_series_report:
    print(f"{timestamp}\t{total_space}")


### Task 5: BGP Update Reports 
Write a PyBGPStream script that analyzes BGP updates for AS59919 during the period from 03-02-2024 13:00 to 13:59:59, focusing specifically on IPv4 updates from collectors 'rrc00' and 'rrc01'. The script needs to report the total number of changes in the Forwarding Information Base (FIB). Break down the changes into three main categories: announcements, withdrawals, and path changes as well as classifying them by their IPv4 prefix lengths.